1.0 PRÁCTICA

1.1 Extracción de datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# CSV ya arreglado: separado por comas
data = pd.read_csv('data/drugsComTest_raw.csv')

data.head()

data.info()

<class 'pandas.DataFrame'>
RangeIndex: 53766 entries, 0 to 53765
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   53766 non-null  int64  
 1   drugName     53766 non-null  str    
 2   condition    53471 non-null  str    
 3   review       53766 non-null  str    
 4   rating       53766 non-null  float64
 5   date         53766 non-null  str    
 6   usefulCount  53766 non-null  int64  
dtypes: float64(1), int64(2), str(4)
memory usage: 2.9 MB


1.2 Trato de datos NULL

In [ ]:
#Tabla rating
# 0) Comprobación rápida antes de limpiar
print("dtype rating (antes):", data["rating"].dtype)
print("nulos rating (antes):", data["rating"].isna().sum())

# 1) Forzar conversión a numérico (lo no-convertible pasa a NaN)
data["rating"] = pd.to_numeric(data["rating"], errors="coerce")

# 2) Contar los NaN (después de convertir)
print(f"La suma de valores nulos en rating es: {data['rating'].isna().sum()}")

# 3) Check de rango (ratings deberían estar entre 0 y 10)
print("min rating:", data["rating"].min(), "max rating:", data["rating"].max())

print(f"La mediana de los ratings es: {data['rating'].median()}")
#Como rating no es nuestra variavle explicita y no sesgará el resultado la vamos a modificar
#Utilizaremos la mediana para rellenar los valores nulos 
data["rating"] = data["rating"].fillna(data["rating"].median())



La suma de valores nulos en rating es: 0
La mediana de los ratings es: 8.0


In [ ]:
# Tabla drugName
# 0) Comprobación rápida antes de limpiar
print("dtype drugName (antes):", data["drugName"].dtype)
print("nulos drugName (antes):", data["drugName"].isna().sum())

# 1) Asegurar tipo string
data["drugName"] = data["drugName"].astype(str)
data["drugName"] = data["drugName"].replace("", np.nan).replace("nan", np.nan)

# 2) Contar los NaN (después de limpiar)
print(f"La suma de valores nulos en drugName es: {data['drugName'].isna().sum()}")

# 3) Moda para rellenar nulos
print(f"La moda de drugName es: {data['drugName'].mode().iloc[0]}")
data["drugName"] = data["drugName"].fillna(data["drugName"].mode().iloc[0])

In [ ]:
# Tabla condition
# 0) Comprobación rápida antes de limpiar
print("dtype condition (antes):", data["condition"].dtype)
print("nulos condition (antes):", data["condition"].isna().sum())

# 1) Asegurar tipo string
data["condition"] = data["condition"].astype(str)
data["condition"] = data["condition"].replace("", np.nan).replace("nan", np.nan)

# 2) Contar los NaN (después de limpiar)
print(f"La suma de valores nulos en condition es: {data['condition'].isna().sum()}")

# 3) Moda para rellenar nulos
print(f"La moda de condition es: {data['condition'].mode().iloc[0]}")
data["condition"] = data["condition"].fillna(data["condition"].mode().iloc[0])

In [ ]:
# Tabla review
# 0) Comprobación rápida antes de limpiar
print("dtype review (antes):", data["review"].dtype)
print("nulos review (antes):", data["review"].isna().sum())

# 1) Asegurar tipo string
data["review"] = data["review"].astype(str)
data["review"] = data["review"].replace("", np.nan).replace("nan", np.nan)

# 2) Contar los NaN (después de limpiar)
print(f"La suma de valores nulos en review es: {data['review'].isna().sum()}")

# 3) Rellenar nulos con placeholder (la moda en textos largos puede no ser representativa)
data["review"] = data["review"].fillna("(No review)")

In [ ]:
# Tabla date
# 0) Comprobación rápida antes de limpiar
print("dtype date (antes):", data["date"].dtype)
print("nulos date (antes):", data["date"].isna().sum())

# 1) Convertir a datetime (lo no convertible pasa a NaT)
data["date"] = pd.to_datetime(data["date"], errors="coerce")

# 2) Contar los NaN (después de convertir)
print(f"La suma de valores nulos en date es: {data['date'].isna().sum()}")

# 3) Rellenar nulos con la moda (fecha más frecuente)
print(f"La moda de date es: {data['date'].mode().iloc[0]}")
data["date"] = data["date"].fillna(data["date"].mode().iloc[0])

In [ ]:
# Tabla usefulCount
# 0) Comprobación rápida antes de limpiar
print("dtype usefulCount (antes):", data["usefulCount"].dtype)
print("nulos usefulCount (antes):", data["usefulCount"].isna().sum())

# 1) Forzar conversión a numérico (lo no-convertible pasa a NaN)
data["usefulCount"] = pd.to_numeric(data["usefulCount"], errors="coerce")

# 2) Contar los NaN (después de convertir)
print(f"La suma de valores nulos en usefulCount es: {data['usefulCount'].isna().sum()}")

# 3) Check de rango (conteos no negativos)
print("min usefulCount:", data["usefulCount"].min(), "max usefulCount:", data["usefulCount"].max())

print(f"La mediana de usefulCount es: {data['usefulCount'].median()}")
data["usefulCount"] = data["usefulCount"].fillna(data["usefulCount"].median())

### 2. Exploración de datos (EDA)

Análisis y visualización del dataset tras el tratamiento de nulos.

#### 2.1 Tamaño del dataset

In [ ]:
n_filas, n_cols = data.shape
memoria_mb = data.memory_usage(deep=True).sum() / 1024**2

print(f"Número de filas: {n_filas:,}")
print(f"Número de columnas: {n_cols}")
print(f"Uso de memoria (aprox.): {memoria_mb:.2f} MB")
print(f"Total de valores (celdas): {n_filas * n_cols:,}")

#### 2.2 Tipos de variables

In [ ]:
tipos = data.dtypes
print("Tipos por columna:")
print(tipos)
print()

# Número de variables numéricas vs categóricas/texto
numericas = (tipos == "int64") | (tipos == "float64")
categoricas = (tipos == "object") | (tipos == "str")
print(f"Variables numéricas: {numericas.sum()} (rating, usefulCount, Unnamed: 0)")
print(f"Variables categóricas/texto: {categoricas.sum()} (drugName, condition, review, date)")

# Gráfico sencillo: número de columnas por tipo
data.dtypes.value_counts().plot(kind="bar", color="steelblue", edgecolor="black")
plt.title("Número de columnas por tipo de datos")
plt.xlabel("Tipo (dtype)")
plt.ylabel("Número de columnas")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# Interpretación: tenemos mayormente texto (reviews, nombres, condiciones, fechas) y algunas numéricas
# (rating, usefulCount) relevantes para modelado.

#### 2.3 Distribución de la variable objetivo (rating)

Consideramos **rating** (1–10) como variable objetivo (ej.: predicción de valoración o clasificación).

In [ ]:
# Distribución de rating
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Histograma
axes[0].hist(data["rating"].dropna(), bins=range(1, 12), align="left", color="steelblue", edgecolor="black")
axes[0].set_xlabel("Rating (1-10)")
axes[0].set_ylabel("Frecuencia")
axes[0].set_title("Histograma de la variable rating")
axes[0].set_xticks(range(1, 11))

# Diagrama de barras (conteo por valor)
data["rating"].value_counts().sort_index().plot(kind="bar", ax=axes[1], color="coral", edgecolor="black")
axes[1].set_xlabel("Rating (1-10)")
axes[1].set_ylabel("Número de reviews")
axes[1].set_title("Conteo por valor de rating")
plt.tight_layout()
plt.show()

print("Estadísticos descriptivos de rating:")
print(data["rating"].describe())

# Interpretación: la distribución de rating está sesgada hacia valores altos (8–10);
# hay menos reviews con valoraciones bajas. Es típico en reviews de productos.

#### 2.4 Posible desbalanceo de clases

Si tratamos **rating** como variable objetivo (por clases), comprobamos si las clases están desbalanceadas.

In [ ]:
# Desbalanceo: proporción de cada clase (rating 1-10)
counts = data["rating"].value_counts().sort_index()
proporciones = (counts / len(data) * 100).round(1)

print("Conteo y porcentaje por clase (rating):")
print(pd.DataFrame({"conteo": counts, "porcentaje (%)": proporciones}))
print()

# Ratio entre la clase más frecuente y la menos frecuente
max_c = counts.max()
min_c = counts.min()
ratio = max_c / min_c
print(f"Clase más frecuente: {counts.idxmax()} (n={max_c})")
print(f"Clase menos frecuente: {counts.idxmin()} (n={min_c})")
print(f"Ratio máx/min: {ratio:.1f}x → hay desbalanceo notable.")

# Gráfico de barras con porcentajes
proporciones.plot(kind="bar", color="teal", edgecolor="black", figsize=(10, 4))
plt.title("Porcentaje de muestras por clase (rating) — desbalanceo")
plt.xlabel("Rating")
plt.ylabel("% del total")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# Interpretación: las clases no están equilibradas; los ratings 8–10 dominan.
# Para clasificación convendría estrategias como muestreo (oversampling/undersampling),
# pesos por clase o métricas adecuadas (F1, AUC) en lugar de solo accuracy.

### 3. Preparación de datos

- Separar variables de entrada (X) y variable objetivo (y).
- Dividir los datos en entrenamiento y test.
- Explicar por qué esta separación es necesaria.

#### 3.1 Separación de X (entrada) e y (objetivo)

La variable objetivo es **rating**. Las variables de entrada son el resto de columnas que usaremos como características (drugName, condition, review, date, usefulCount). Excluimos `Unnamed: 0` por ser solo un índice.

In [ ]:
# Variable objetivo
y = data["rating"]

# Variables de entrada (todas las columnas excepto la objetivo y el índice)
columnas_entrada = ["drugName", "condition", "review", "date", "usefulCount"]
X = data[columnas_entrada]

print("Variable objetivo (y):")
print(f"  Forma: {y.shape}, nombre: rating")
print()
print("Variables de entrada (X):")
print(f"  Forma: {X.shape}")
print(f"  Columnas: {list(X.columns)}")
X.head()

#### 3.2 División en entrenamiento y test

Usamos `train_test_split` con un 80 % para entrenamiento y 20 % para test, y `random_state` fijo para reproducibilidad.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Tamaños tras la división:")
print(f"  Entrenamiento: X_train {X_train.shape}, y_train {y_train.shape}")
print(f"  Test:         X_test {X_test.shape}, y_test {y_test.shape}")
print()
print(f"  Proporción train: {len(X_train) / len(X):.1%}, test: {len(X_test) / len(X):.1%}")

#### 3.3 ¿Por qué es necesaria esta separación?

- **Evitar sobreajuste (overfitting):** Si entrenamos y evaluamos con los mismos datos, el modelo puede “memorizar” y dar métricas optimistas que no se mantienen con datos nuevos. Con un conjunto de **test** que el modelo no ha visto, la evaluación es más fiable.

- **Simular uso real:** En la práctica el modelo predecirá sobre datos futuros. El conjunto de test actúa como sustituto de esos datos futuros para estimar cómo se comportará el modelo.

- **Buenas prácticas:** Separar train/test (y más adelante validación si hace falta) es estándar en aprendizaje supervisado y permite comparar modelos de forma justa.